**Connect google drive**

In [11]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Dec 14 12:22:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 CV_DR.pdf  'My Drive'	 Video	 yolov3


**1) Clone the Darknet**



In [13]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [14]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variab

**3) Configure Darknet network for training YOLO V3**

In [15]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [16]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [17]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [18]:
!echo "human" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [19]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-12-14 12:25:01--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.0MB/s    in 20s     

2023-12-14 12:29:23 (7.63 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [20]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/images/01.jpg  
  inflating: data/obj/images/01.txt  
  inflating: data/obj/images/02.jpg  
  inflating: data/obj/images/02.txt  
  inflating: data/obj/images/03.jpg  
  inflating: data/obj/images/03.txt  
  inflating: data/obj/images/04.jpg  
  inflating: data/obj/images/04.txt  
  inflating: data/obj/images/05.jpg  
  inflating: data/obj/images/05.txt  
  inflating: data/obj/images/06.jpg  
  inflating: data/obj/images/06.txt  
  inflating: data/obj/images/07.jpeg  
  inflating: data/obj/images/07.txt  
  inflating: data/obj/images/08.jpg  
  inflating: data/obj/images/08.txt  
  inflating: data/obj/images/09.jpg  
  inflating: data/obj/images/09.txt  
  inflating: data/obj/images/10.jpg  
  inflating: data/obj/images/10.txt  
  inflating: data/obj/images/100.jpg  
  inflating: data/obj/images/100.txt  
  inflating: data/obj/images/11.jpg  
  inflating: data/obj/images/11.txt  
  inflating: data/obj/images/12.jpg  
  inflatin

In [21]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [22]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/34.jpg', 'data/obj/images/88.jpg', 'data/obj/images/66.jpg', 'data/obj/images/23.jpg', 'data/obj/images/53.jpg', 'data/obj/images/18.jpg', 'data/obj/images/57.jpg', 'data/obj/images/11.jpg', 'data/obj/images/39.jpg', 'data/obj/images/92.jpg', 'data/obj/images/02.jpg', 'data/obj/images/12.jpg', 'data/obj/images/61.jpg', 'data/obj/images/82.jpg', 'data/obj/images/32.jpg', 'data/obj/images/20.jpg', 'data/obj/images/10.jpg', 'data/obj/images/93.jpg', 'data/obj/images/16.jpg', 'data/obj/images/50.jpg', 'data/obj/images/76.jpg', 'data/obj/images/74.jpg', 'data/obj/images/85.jpg', 'data/obj/images/44.jpg', 'data/obj/images/01.jpg', 'data/obj/images/91.jpg', 'data/obj/images/83.jpg', 'data/obj/images/64.jpg', 'data/obj/images/55.jpg', 'data/obj/images/40.jpg', 'data/obj/images/33.jpg', 'data/obj/images/78.jpg', 'data/obj/images/65.jpg', 'data/obj/images/96.jpg', 'data/obj/images/100.jpg', 'data/obj/images/45.jpg', 'data/obj/images/95.jpg', 'data/obj/images/47.jpg', 'data/obj/

In [23]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 132208, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.925798), count: 4, class_loss = 0.000100, iou_loss = 0.018632, total_loss = 0.018732 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 132212, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.825730), count: 4, class_loss = 0.014187, iou_loss = 0.082017, total_loss = 0.096204 
v3 (m